In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
url = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(url)o

In [5]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

x_train = train_set.drop('median_house_value', axis=1)
y = train_set['median_house_value'].copy()

x_num = x_train.drop('ocean_proximity', axis=1)

In [6]:
# Pipeline quramiz

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

#bizga kerakli ustunlar
rooms_ix, bedrooms_ix, population_ix,households_ix = 3,4,5,6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self,x, y=None):
        return self # bu funksiya faqat transformer. estimator emas
    def transform(self,x):
        rooms_per_household = x[:, rooms_ix]/x[:,households_ix]
        population_per_household = x[:,population_ix]/x[:,households_ix]
        if self.add_bedrooms_per_room: # bu ixtiyoriy ustun boladi
            bedrooms_per_room = x[:, bedrooms_ix]/x[:, rooms_ix]
            return np.c_[x, rooms_per_household,  population_per_household, bedrooms_per_room]
        else:
            return np.c_[x, rooms_per_household,  population_per_household]

In [8]:
# sonli ustunlar uchun

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

In [10]:
# matnli ustunlar uchun

In [13]:
from sklearn.compose import ColumnTransformer

num_attribs = list(x_num) # matnli ustunlar nomini sugurib oldik
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    
    ('num',num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

In [14]:
x_prepared = full_pipeline.fit_transform(x_train)

In [16]:
x_prepared[:5]

array([[ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
         0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
        -0.2117846 ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.70916212, -0.87669601,  1.61811813,  0.34029326,  0.59309419,
        -0.09890135,  0.6720272 , -0.03584338, -0.40283542, -0.11736222,
         0.34218528,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.44760309, -0.46014647, -1.95271028, -0.34259695, -0.49522582,
        -0.44981806, -0.43046109,  0.14470145,  0.08821601, -0.03227969,
        -0.66165785,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 1.23269811, -1.38217186,  0.58654547, -0.56148971, -0.40930582,
        -0.00743434, -0.38058662, -1.01786438, -0.60001532,  0.07750687,
         0.78303162,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.10855122,  0.5320839 ,  1

In [17]:
### LINEAR REGRESSION

In [18]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()

In [19]:
lr_model.fit(x_prepared, y)

LinearRegression()

In [21]:
test_data = x_train.sample(10)
test_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
4780,-118.32,34.04,42.0,1766.0,404.0,1117.0,367.0,2.0259,<1H OCEAN
10669,-117.86,33.62,23.0,3166.0,411.0,1092.0,345.0,7.9367,<1H OCEAN
2504,-120.45,36.86,34.0,673.0,173.0,539.0,182.0,2.3523,INLAND
3456,-118.42,34.32,30.0,3027.0,604.0,1970.0,590.0,4.3409,<1H OCEAN
9762,-121.73,36.50,27.0,3184.0,520.0,1121.0,493.0,5.6383,<1H OCEAN
7096,-118.00,33.93,24.0,4534.0,967.0,2547.0,895.0,3.9575,<1H OCEAN
162,-122.24,37.81,52.0,2513.0,502.0,1048.0,518.0,3.6750,NEAR BAY
11988,-117.51,34.00,36.0,3791.0,746.0,2258.0,672.0,3.2067,INLAND
4654,-118.33,34.05,48.0,2405.0,527.0,1868.0,502.0,3.3750,<1H OCEAN
570,-122.24,37.72,5.0,18634.0,2885.0,7427.0,2718.0,7.6110,NEAR BAY


In [23]:
test_label = y.loc[test_data.index]
test_label

4780     168800.0
10669    500001.0
2504      66000.0
3456     156000.0
9762     377000.0
7096     215400.0
162      269900.0
11988    124700.0
4654     257800.0
570      350700.0
Name: median_house_value, dtype: float64

In [24]:
test_data_prepared = full_pipeline.transform(test_data)
test_data_prepared

array([[ 6.29385213e-01, -7.50327052e-01,  1.06265593e+00,
        -4.02838445e-01, -3.20999155e-01, -2.72160977e-01,
        -3.49086953e-01, -9.74067293e-01, -2.61066999e-01,
        -4.60896966e-03,  2.74299024e-01,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 8.58743834e-01, -9.46900993e-01, -4.45027177e-01,
         2.40963773e-01, -3.04292488e-01, -2.94148239e-01,
        -4.06836340e-01,  2.12995931e+00,  1.56728209e+00,
         5.89513533e-03, -1.43190146e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-4.32644923e-01,  5.69526554e-01,  4.27841989e-01,
        -9.05464033e-01, -8.72319162e-01, -7.80506485e-01,
        -8.34706795e-01, -8.02659991e-01, -7.27780734e-01,
        -1.16961591e-02,  7.62152173e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 5.79524643e-01, -6.19277757e

In [25]:
predicted_labels = lr_model.predict(test_data_prepared)

In [26]:
predicted_labels

array([160771.90110863, 370603.00518694, 118120.81479317, 225953.407496  ,
       328857.57894001, 223018.28137155, 258598.37963505, 135320.24799215,
       207544.33487663, 411157.45952554])

In [28]:
pd.DataFrame({'bashorat':predicted_labels,'asl qiymati':test_label})

,bashorat,asl qiymati
4780,160771.901109,168800.0
10669,370603.005187,500001.0
2504,118120.814793,66000.0
3456,225953.407496,156000.0
9762,328857.578940,377000.0
7096,223018.281372,215400.0
162,258598.379635,269900.0
11988,135320.247992,124700.0
4654,207544.334877,257800.0
570,411157.459526,350700.0


In [29]:
####### 2-qism test qismi

In [30]:
test_set.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,47700.0,INLAND
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,45800.0,INLAND
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,500001.0,NEAR BAY
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,218600.0,<1H OCEAN
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,278000.0,NEAR OCEAN


In [32]:
x_test = test_set.drop('median_house_value', axis=1)

y_test = test_set['median_house_value'].copy()

In [33]:
x_test_prepared = full_pipeline.transform(x_test)

In [35]:
y_predicted = lr_model.predict(x_test_prepared)

In [36]:
pd.DataFrame({'bashorat':y_predicted,'asl qiymati':y_test})

,bashorat,asl qiymati
20046,61874.254601,47700.0
3024,121853.525111,45800.0
15663,267770.943681,500001.0
20484,264468.298380,218600.0
9814,258485.695855,278000.0
...,...,...
15362,214774.565911,263300.0
16623,236868.742582,266800.0
18086,447837.046479,500001.0
2144,117275.921461,72300.0


In [37]:
###### tekshirib koramiz
#

In [38]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_predicted)

print("MAE=", mae)

MAE= 50898.73953494079


In [39]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_predicted)

print("RMSE=", np.sqrt(mse)) # np.sqrt(mse) qilganimiz sababi kvadratdan chiqazish kk edi rmse natijani olish uchun

RMSE= 72701.32600762135


In [40]:
# RANDOM_FOREST

In [41]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor()

rf_model.fit(x_prepared, y)

RandomForestRegressor()

In [42]:
y_predicted = rf_model.predict(x_test_prepared)

In [43]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_predicted)

print("RMSE=", np.sqrt(mse)) # np.sqrt(mse) qilganimiz sababi kvadratdan chiqazish kk edi rmse natijani olish uchun

RMSE= 50187.17086684911


In [44]:
# CROSS_VALIDATION

In [47]:
x = df.drop('median_house_value',axis=1)
y = df['median_house_value'].copy()

x_prepared = full_pipeline.transform(x)

In [48]:
from sklearn.model_selection import cross_val_score

cross_val_score(lr_model, x_prepared, y, scoring="neg_mean_squared_error", cv=5)


array([-5.38681502e+09, -5.59717065e+09, -5.68997624e+09, -5.86890635e+09,
       -4.38197413e+09])

In [51]:
mse_scores = cross_val_score(lr_model, x_prepared, y, scoring="neg_mean_squared_error", cv=5)



In [53]:
def display_scores(scores):
    print('scores:', scores)
    print('Mean:', scores.mean())
    print('std.dev', scores.std())

In [54]:
display_scores(np.sqrt(-mse_scores))

scores: [73394.92502922 74814.24096819 75431.93119241 76608.78768825
 66196.48128669]
Mean: 73289.27323295095
std.dev 3694.7136787223762


In [56]:
scores = cross_val_score(rf_model, x_prepared, y, scoring='neg_mean_squared_error',cv=10)

lr_rmse_scores = np.sqrt(-scores)
display_scores(lr_rmse_scores)

scores: [97116.09130572 47424.73387653 65524.07926539 56364.52328054
 61336.9342455  59912.85712871 46878.38998078 78773.48014988
 74383.0887085  49328.23763982]
Mean: 63704.24155813873
std.dev 15130.87765315829
